# Memory

In many real-world scenarios we do not have access to the ground truth state of the environment. In these cases we only get observations that have some mutual information with the true state. In this case the Markov property is violated (we are dealing with a  [POMDP](https://en.wikipedia.org/wiki/Partially_observable_Markov_decision_process>) and hence information from past observations influences the estimate of the current state. Because the belief about the current state is dependent on previous observations, also the future behavior and action selection decision are dependent on them.

To respect this in our policy, it needs to be able to reason about sequences of observations. A natural way to implement this is using recurrent neural networs (RNNs) that carry an internal state that can transport information through time.

RLtools initially only supports the [GRU (Gated Recurrent Unit)](https://en.wikipedia.org/wiki/Gated_recurrent_unit), a widely used and time-tested RNN architecture.

In [1]:
#include <vector>
#include <algorithm>
#include <numeric>
#include <iostream>

#define RL_TOOLS_BACKEND_ENABLE_OPENBLAS
#include <rl_tools/operations/cpu_mux.h>
#include <rl_tools/nn/optimizers/adam/instance/operations_generic.h>
#include <rl_tools/nn/operations_cpu_mux.h>
#include <rl_tools/nn/layers/gru/operations_generic.h>
#include <rl_tools/nn_models/sequential/operations_generic.h>
#include <rl_tools/nn/optimizers/adam/operations_generic.h>
#include <rl_tools/nn/loss_functions/mse/operations_generic.h>
namespace rlt = rl_tools;
#pragma cling load("openblas")

In [2]:
using T = float;
using DEVICE = rlt::devices::DEVICE_FACTORY<rlt::devices::DefaultCPUSpecification>;
using RNG = DEVICE::SPEC::RANDOM::ENGINE<>;
using TI = typename DEVICE::index_t;
constexpr bool DYNAMIC_ALLOCATION = true;

In [3]:
constexpr TI SEQUENCE_LENGTH = 10;
constexpr TI BATCH_SIZE = 10;
constexpr TI INPUT_DIM = 1;
constexpr TI HIDDEN_DIM = 8;
constexpr TI OUTPUT_DIM = 1;
using INPUT_SHAPE = rlt::tensor::Shape<TI, SEQUENCE_LENGTH, BATCH_SIZE, INPUT_DIM>;
using GRU_CONFIG = rlt::nn::layers::gru::Configuration<T, TI, HIDDEN_DIM, rlt::nn::parameters::groups::Normal, DYNAMIC_ALLOCATION>;
using GRU = rlt::nn::layers::gru::BindConfiguration<GRU_CONFIG>;
using OUTPUT_LAYER_CONFIG = rlt::nn::layers::dense::Configuration<T, TI, OUTPUT_DIM, rlt::nn::activation_functions::IDENTITY>;
using OUTPUT_LAYER = rlt::nn::layers::dense::BindConfiguration<OUTPUT_LAYER_CONFIG>;

In [4]:
template <typename T_CONTENT, typename T_NEXT_MODULE = rlt::nn_models::sequential::OutputModule>
using Module = typename rlt::nn_models::sequential::Module<T_CONTENT, T_NEXT_MODULE>;

using MODULE_CHAIN = Module<GRU, Module<OUTPUT_LAYER>>;
using CAPABILITY = rlt::nn::capability::Gradient<rlt::nn::parameters::Adam>;
using MODEL = rlt::nn_models::sequential::Build<CAPABILITY, MODULE_CHAIN, INPUT_SHAPE>;

In [5]:
struct ADAM_PARAMS: rlt::nn::optimizers::adam::DEFAULT_PARAMETERS_TENSORFLOW<T>{
    static constexpr T ALPHA = 0.003;
};
using ADAM_SPEC = rlt::nn::optimizers::adam::Specification<T, TI, ADAM_PARAMS>;
using OPTIMIZER = rlt::nn::optimizers::Adam<ADAM_SPEC>;


In [6]:
DEVICE device;
RNG rng;
MODEL model;
MODEL::Buffer<> buffer;
MODEL::State<> state;
OPTIMIZER optimizer;
rlt::Tensor<rlt::tensor::Specification<T, TI, MODEL::INPUT_SHAPE>> input;
rlt::Tensor<rlt::tensor::Specification<T, TI, MODEL::OUTPUT_SHAPE>> output_target, d_output;
constexpr TI DATASET_SIZE = 1000;
using DATASET_SHAPE = rlt::tensor::Shape<TI, DATASET_SIZE, SEQUENCE_LENGTH, INPUT_DIM>;
using DATASET_TARGET_SHAPE = rlt::tensor::Shape<TI, DATASET_SIZE, SEQUENCE_LENGTH, OUTPUT_DIM>;
rlt::Tensor<rlt::tensor::Specification<T, TI, DATASET_SHAPE>> dataset_X;
rlt::Tensor<rlt::tensor::Specification<T, TI, DATASET_TARGET_SHAPE>> dataset_y;

rlt::init(device);
rlt::malloc(device, rng);
constexpr TI SEED = 0;
rlt::init(device, rng, SEED);
rlt::malloc(device, model);
rlt::malloc(device, buffer);
rlt::malloc(device, state);
rlt::malloc(device, optimizer);
rlt::malloc(device, input);
rlt::malloc(device, output_target);
rlt::malloc(device, d_output);
rlt::malloc(device, dataset_X);
rlt::malloc(device, dataset_y);
rlt::init_weights(device, model, rng);
rlt::reset_optimizer_state(device, optimizer, model);

In [7]:
template <typename DATASET_X, typename DATASET_Y>
void max_dataset(DATASET_X& dataset_X, DATASET_Y& dataset_y){
    static_assert(DATASET_X::SHAPE::FIRST == DATASET_Y::SHAPE::FIRST);
    static_assert(DATASET_X::SHAPE::template GET<1> == DATASET_Y::SHAPE::template GET<1>);
    rlt::randn(device, dataset_X, rng);
    for(TI sample_i = 0; sample_i < DATASET_X::SHAPE::FIRST; sample_i++){
        T max;
        bool max_set = false;
        for(TI step_i = 0; step_i < DATASET_X::SHAPE::template GET<1>; step_i++){
            T el = rlt::get(device, dataset_X, sample_i, step_i, 0);
            if(!max_set || el > max){
                max_set = true;
                max = el;
            }
            rlt::set(device, dataset_y, max, sample_i, step_i, 0);
        }
    }
}

In [8]:
max_dataset(dataset_X, dataset_y);

In [9]:
rlt::print(device, rlt::view(device, dataset_X, 0));
rlt::print(device, rlt::view(device, dataset_y, 0));

  -1.507621e+00
   1.071986e+00
   8.269271e-01
   1.601774e+00
  -1.074195e+00
  -5.420533e-01
  -6.830205e-01
   1.492320e+00
   6.583855e-02
   9.513746e-01

  -1.507621e+00
   1.071986e+00
   1.071986e+00
   1.601774e+00
   1.601774e+00
   1.601774e+00
   1.601774e+00
   1.601774e+00
   1.601774e+00
   1.601774e+00



In [ ]:
std::vector<TI> indices(DATASET_SIZE);
std::iota(indices.begin(), indices.end(), 0); // fill with range 0..DATASET_SIZE
constexpr TI N_EPOCH = 1000;
for(TI epoch_i = 0; epoch_i < N_EPOCH; epoch_i++){
    T epoch_loss = 0;
    std::shuffle(indices.begin(), indices.end(), rng.engine);
    for(TI batch_i = 0; batch_i < DATASET_SIZE / BATCH_SIZE; batch_i++){
        for(TI sequence_i = 0; sequence_i < BATCH_SIZE; sequence_i++){
            TI index = BATCH_SIZE * batch_i + sequence_i;
            auto input_sample = rlt::view(device, input, sequence_i, rlt::tensor::ViewSpec<1>{});
            auto output_sample = rlt::view(device, output_target, sequence_i, rlt::tensor::ViewSpec<1>{});
            auto dataset_input_sample = rlt::view(device, dataset_X, indices[index], rlt::tensor::ViewSpec<0>{});
            auto dataset_output_sample = rlt::view(device, dataset_y, indices[index], rlt::tensor::ViewSpec<0>{});
            rlt::copy(device, device, dataset_input_sample, input_sample);
            rlt::copy(device, device, dataset_output_sample, output_sample);
        }
        rlt::forward(device, model, input, buffer, rng);
        auto output = rlt::output(device, model);
        auto output_matrix_view = rlt::matrix_view(device, output);
        auto output_target_matrix_view = rlt::matrix_view(device, output_target);
        auto d_output_matrix_view = rlt::matrix_view(device, d_output);
        rlt::nn::loss_functions::mse::gradient(device, output_matrix_view, output_target_matrix_view, d_output_matrix_view);
        T batch_loss = rlt::nn::loss_functions::mse::evaluate(device, output_matrix_view, output_target_matrix_view); 
        epoch_loss += batch_loss;
        rlt::zero_gradient(device, model);
        rlt::backward(device, model, input, d_output, buffer);
        rlt::step(device, optimizer, model);
    }
    if(epoch_i % 20 == 0){
        std::cout << "Epoch " << epoch_i << " loss: " << epoch_loss << std::endl;
    }
}

Epoch 0 loss: 8.599841e+01
Epoch 20 loss: 6.585799e-01
Epoch 40 loss: 2.175719e-01
Epoch 60 loss: 1.164413e-01
